### 데이터 정제

In [5]:
library(tidyverse)

In [6]:
options(warn = -1)

In [7]:
load("./03_integrated/03_apt_price.rdata")

In [8]:
apt_price %>% head

,X,year,month,day,price,code,dong_nm,jibun,con_year,apt_nm,area,floor
,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<dbl>,<chr>
1,1,2021,1,5,"31,000",11680,역삼동,720-25,2002,대우디오빌,30.03,4
2,2,2021,1,6,"61,000",11680,역삼동,766-8,2002,트레벨,33.48,3
3,3,2021,1,7,"198,000",11680,역삼동,757,2005,역삼래미안,59.73,14
4,4,2021,1,7,"193,800",11680,역삼동,757,2005,역삼래미안,59.40,16
5,5,2021,1,9,"91,000",11680,역삼동,796-29,2012,강남서해더블루,66.04,10
6,6,2021,1,11,"80,000",11680,역삼동,783-3,2002,갤러리하우스,84.95,1


- 결측값 확인

In [9]:
apt_price %>% dim
colSums(is.na(apt_price))

[1] 43404    12

X     year    month      day    price     code  dong_nm    jibun 
       0        0        0        0        0        0        0        0 
con_year   apt_nm     area    floor 
       0        0        0       12

- 결측값 제거

In [10]:
apt_price2 <- na.omit(apt_price)
dim(apt_price2)
colSums(is.na(apt_price2))

[1] 43392    12

X     year    month      day    price     code  dong_nm    jibun 
       0        0        0        0        0        0        0        0 
con_year   apt_nm     area    floor 
       0        0        0        0

### 
### 데이터 형식 수정

In [11]:
glimpse(apt_price2)

Rows: 43,392
Columns: 12
$ X        <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18~
$ year     <int> 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2~
$ month    <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1~
$ day      <int> 5, 6, 7, 7, 9, 11, 11, 11, 12, 13, 14, 15, 16, 16, 16, 18, 18~
$ price    <chr> "      31,000", "      61,000", "     198,000", "     193,800~
$ code     <int> 11680, 11680, 11680, 11680, 11680, 11680, 11680, 11680, 11680~
$ dong_nm  <chr> " 역삼동", " 역삼동", " 역삼동", " 역삼동", " 역삼동", " 역삼~
$ jibun    <chr> "720-25", "766-8", "757", "757", "796-29", "783-3", "754-1", ~
$ con_year <int> 2002, 2002, 2005, 2005, 2012, 2002, 2006, 2006, 2006, 2012, 2~
$ apt_nm   <chr> "대우디오빌", "트레벨", "역삼래미안", "역삼래미안", "강남서해~
$ area     <dbl> 30.0300, 33.4800, 59.7300, 59.4000, 66.0400, 84.9500, 84.9097~
$ floor    <chr> "4", "3", "14", "16", "10", "1", "7", "16", "10", "7", "8", "~


#### 1. price는 공백이 포함된 문자형 => 수정 후 숫자형으로 변경

In [12]:
head(apt_price2$price)

[1] "      31,000" "      61,000" "     198,000" "     193,800" "      91,000"
[6] "      80,000"

#### 2. 아파트 이름 뒤에 괄호 안에 불필요한 정보가 포함 => **괄호 '('부터 시작하는 문자 제거**

In [13]:
apt_price2$apt_nm[21:30]

[1] "대우디오빌"                "대우디오빌플러스"         
 [3] "강남서해더블루"            "로얄빌리지"               
 [5] "역삼동하나빌"              "역삼푸르지오"             
 [7] "래미안그레이튼(진달래2차)" "이스턴오피스텔"           
 [9] "e-편한세상"                "쌍용플레티넘밸류"

#### 3. year, month, day를 조합하여 날짜형 변수 date 생성

#### 위의 것들 일괄 적용
#### ```gsub("변경할  text(패턴)", "대체할 text", text객체)``` 

- ```<.*?>``` : < ~ > 안의 모든 내용
- ```\\(.*","``` : '('부터 시작하는 문자

In [15]:
library(lubridate)

In [16]:
apt_price3 <- apt_price2 %>% 
    mutate(price = str_trim(price),
           ymd = make_date(year, month, day),
           ym = make_date(year = year, month = month)) %>%
    mutate(price = str_replace_all(price, ",", "") %>% as.numeric()) %>%
    mutate(apt_nm = gsub("\\(.*","", apt_nm)) 

In [17]:
apt_price3 %>% head(3) %>% print

  X year month day  price  code dong_nm  jibun con_year     apt_nm  area floor
1 1 2021     1   5  31000 11680  역삼동 720-25     2002 대우디오빌 30.03     4
2 2 2021     1   6  61000 11680  역삼동  766-8     2002     트레벨 33.48     3
3 3 2021     1   7 198000 11680  역삼동    757     2005 역삼래미안 59.73    14
         ymd         ym
1 2021-01-05 2021-01-01
2 2021-01-06 2021-01-01
3 2021-01-07 2021-01-01


#### 
#### 주소 조합
- 시군구 정보를 (sigun_code.csv) 통하여, 주소를 조합한 변수 'juso_jibun'생성

In [18]:
loc <- read.csv("./01_code/sigun_code/sigun_code.csv", fileEncoding = "UTF-8")

In [19]:
loc %>% head(2) %>% print

   code       sido sigungu    addr_1            addr_2
1 11110 서울특별시  종로구 서울_종로 서울특별시 종로구
2 11140 서울특별시    중구 서울_중구   서울특별시 중구


In [30]:
apt_price4 <- apt_price3 %>% merge(loc, by = 'code') %>%
    mutate(juso_jibun = paste0(addr_2, dong_nm, " ", jibun, " ", apt_nm))

In [31]:
apt_price4 %>% head(3) %>% print

   code X year month day  price dong_nm jibun con_year            apt_nm   area
1 11110 1 2021     1  14 130000  청운동 56-45     2000          청운현대 129.76
2 11110 2 2021     1   7 150000  사직동   9-1     2008  광화문스페이스본 144.52
3 11110 3 2021     1  15 175000  내수동    71     2004 경희궁의아침2단지 174.55
  floor        ymd         ym       sido sigungu    addr_1            addr_2
1     2 2021-01-14 2021-01-01 서울특별시  종로구 서울_종로 서울특별시 종로구
2     6 2021-01-07 2021-01-01 서울특별시  종로구 서울_종로 서울특별시 종로구
3     4 2021-01-15 2021-01-01 서울특별시  종로구 서울_종로 서울특별시 종로구
                                     juso_jibun
1       서울특별시 종로구 청운동 56-45 청운현대
2 서울특별시 종로구 사직동 9-1 광화문스페이스본
3 서울특별시 종로구 내수동 71 경희궁의아침2단지


#### 평당 매매가
- 1평 = 3.3 $m^2$
- 평당 매매가 = (매매가 / 면적) X 3.3
- 평당 매매가를 나타내는 파생변수 'py' 생성

In [32]:
apt_price5 <- apt_price4 %>% 
    mutate(py = (price / area * 3.3) %>% round())
apt_price5 %>% head(2) %>% print

   code X year month day  price dong_nm jibun con_year           apt_nm   area
1 11110 1 2021     1  14 130000  청운동 56-45     2000         청운현대 129.76
2 11110 2 2021     1   7 150000  사직동   9-1     2008 광화문스페이스본 144.52
  floor        ymd         ym       sido sigungu    addr_1            addr_2
1     2 2021-01-14 2021-01-01 서울특별시  종로구 서울_종로 서울특별시 종로구
2     6 2021-01-07 2021-01-01 서울특별시  종로구 서울_종로 서울특별시 종로구
                                     juso_jibun   py
1       서울특별시 종로구 청운동 56-45 청운현대 3306
2 서울특별시 종로구 사직동 9-1 광화문스페이스본 3425


 
#### 층수 변환
- 층수를 나타내는 'floor'변수를 수치형으로 변경

In [33]:
class(apt_price5$floor); head(apt_price5$floor)

[1] "character"

[1] "2"  "6"  "4"  "4"  "13" "13"

In [34]:
apt_price5$floor <- apt_price5$floor %>% as.numeric()

- 아파트 층수는 모두 양수인데, 입력 오류로 인한 음수값이 존재할 수 있음 => 절댓값으로 변환

In [35]:
apt_price5$floor <- apt_price5$floor %>% abs()
apt_price5 %>% dim

[1] 43390    20

#### 카운트 변수 추가
- 모든 거래 건수에 1이라는 숫자 부여 => **이후 분석 과정에서 거래 건수를 집계하기 위함**

In [36]:
apt_price5$cnt <- 1
apt_price5 %>% dim
head(apt_price5, 3) %>% print

[1] 43390    21

   code X year month day  price dong_nm jibun con_year            apt_nm   area
1 11110 1 2021     1  14 130000  청운동 56-45     2000          청운현대 129.76
2 11110 2 2021     1   7 150000  사직동   9-1     2008  광화문스페이스본 144.52
3 11110 3 2021     1  15 175000  내수동    71     2004 경희궁의아침2단지 174.55
  floor        ymd         ym       sido sigungu    addr_1            addr_2
1     2 2021-01-14 2021-01-01 서울특별시  종로구 서울_종로 서울특별시 종로구
2     6 2021-01-07 2021-01-01 서울특별시  종로구 서울_종로 서울특별시 종로구
3     4 2021-01-15 2021-01-01 서울특별시  종로구 서울_종로 서울특별시 종로구
                                     juso_jibun   py cnt
1       서울특별시 종로구 청운동 56-45 청운현대 3306   1
2 서울특별시 종로구 사직동 9-1 광화문스페이스본 3425   1
3 서울특별시 종로구 내수동 71 경희궁의아침2단지 3309   1


### 전처리된 데이터 저장

In [37]:
apt_price <- apt_price5 %>% 
    select(ymd, ym, year, code, addr_1, apt_nm, juso_jibun,
           price, con_year, area, floor, py, cnt)
apt_price %>% head(3) %>% print

         ymd         ym year  code    addr_1            apt_nm
1 2021-01-14 2021-01-01 2021 11110 서울_종로          청운현대
2 2021-01-07 2021-01-01 2021 11110 서울_종로  광화문스페이스본
3 2021-01-15 2021-01-01 2021 11110 서울_종로 경희궁의아침2단지
                                     juso_jibun  price con_year   area floor
1       서울특별시 종로구 청운동 56-45 청운현대 130000     2000 129.76     2
2 서울특별시 종로구 사직동 9-1 광화문스페이스본 150000     2008 144.52     6
3 서울특별시 종로구 내수동 71 경희궁의아침2단지 175000     2004 174.55     4
    py cnt
1 3306   1
2 3425   1
3 3309   1


In [38]:
dir.create('./04_pre_process')
save(apt_price, file = "./04_pre_process/04_preprocess.rdata") 
write.csv(apt_price, "./04_pre_process/04_preprocess.csv") 